In [16]:
import numpy as np
import pandas as pd
# thư viện này để tạo ma trận thưa 
import scipy
import sklearn
from scipy.sparse import csr_matrix
# thư viện này dùng để xây dựng mô hình knn
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import NearestNeighbors

## đọc dữ liệu


In [17]:
column_names1 = ['user id','movie id','rating','timestamp']
dataset = pd.read_csv('Data Movielens/u.data', sep='\t',header=None,names=column_names1)
d = 'movie id | movie title | release date | video release date | IMDb URL | unknown | Action | Adventure | Animation | Children | Comedy | Crime | Documentary | Drama | Fantasy | Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi | Thriller | War | Western'
column_names2 = d.split(' | ')
items_dataset = pd.read_csv('Data Movielens/u.item', sep='|',header=None,names=column_names2,encoding='latin-1')
movie_dataset = items_dataset[['movie id','movie title']]


# tiền xử lý dữ liệu

In [18]:
# B1: hợp nhất 2 dataframe dataset và movie_dataset đã đọc ở trên dựa trên cột chung là movie id
merged_dataset = pd.merge(dataset, movie_dataset, how='inner', on='movie id')

# B2: sử dụng groupby và agg để tính điểm trung bình đánh giá của mỗi người dùng cho mỗi bộ phim.
refined_dataset = merged_dataset.groupby(by=['user id','movie title'], as_index=False).agg({"rating":"mean"})

# B3: Tạo một DataFrame mới chứa số lượng đánh giá cho mỗi điểm đánh giá.
rating_count_df = pd.DataFrame(refined_dataset.groupby(['rating']).size(), columns=['count'])


In [19]:
rating_count_df

,count
rating,
1.0,6083
1.5,3
2.0,11334
2.5,6
3.0,27060
3.5,19
4.0,34042
4.5,16
5.0,21130


In [20]:
# B4: Trong bộ dữ liệu sẽ có các đánh giá bị bỏ trống 
# (người dùng không đánh giá phim này), ta sẽ tính số lượng 
# các đánh giá thiếu này và bổ sung vào rating_count_df
num_users = len(refined_dataset['user id'].value_counts())
num_items = len(refined_dataset['movie title'].value_counts())
total_count = num_items * num_users
zero_count = total_count-refined_dataset.shape[0]
rating_count_df = rating_count_df.append(
    pd.DataFrame({'count': zero_count}, index=[0.0]),
    verify_integrity=True,
).sort_index()

# vì giá trị của số lượng đánh giá lớn, do đó ta giảm đi bằng cách lấy logarit của nó
rating_count_df['log_count'] = np.log(rating_count_df['count'])
rating_count_df = rating_count_df.reset_index().rename(columns={'index': 'rating score'})

C:\Users\LAPTOP\AppData\Local\Temp\ipykernel_5276\3598299161.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rating_count_df = rating_count_df.append(


In [21]:
rating_count_df

,rating score,count,log_count
0,0.0,1469459,14.200405
1,1.0,6083,8.713253
2,1.5,3,1.098612
3,2.0,11334,9.335562
4,2.5,6,1.791759
5,3.0,27060,10.205812
6,3.5,19,2.944439
7,4.0,34042,10.435350
8,4.5,16,2.772589
9,5.0,21130,9.958449


In [22]:
movie_to_user_df = refined_dataset.pivot(
     index='movie title',
     columns='user id',
      values='rating').fillna(0)

#Tạo một ma trận bảng đánh giá của người dùng cho các bộ phim, với các giá trị là điểm đánh giá.
# Chuyển đổi ma trận thành dạng ma trận thưa thớt để giảm lượng dữ liệu không cần thiết.
movie_to_user_sparse_df = csr_matrix(movie_to_user_df.values)

movies_list = list(movie_to_user_df.index)

# Tạo một từ điển với tên phim là khóa và chỉ số của nó từ movies_list:
movie_dict = {movie : index for index, movie in enumerate(movies_list)}
case_insensitive_movies_list = [i.lower() for i in movies_list]

# Xây dựng mô hình

## Điều chỉnh các tham số của thuật toán với <font color=red>GridSearchCV</font> để tìm ra các tham số tốt nhất cho thuật toán

In [23]:
# Xác định lưới tham số
param_grid = {
    'n_neighbors': [5, 10, 15],
    'radius': [1, 2, 3],
    'metric': ['cosine', 'euclidean'],
    'algorithm': ['brute'] 
}

# Tạo mô hình NearestNeighbors
knn_movie_model = NearestNeighbors()

# Tạo đối tượng GridSearchCV
grid_search = GridSearchCV(knn_movie_model, param_grid, scoring='neg_mean_squared_error', cv=5, n_jobs=-1)

#Điều chỉnh tìm kiếm dạng lưới cho phù hợp với dữ liệu
grid_search.fit(movie_to_user_sparse_df)

# In các thông số tốt nhất được tìm thấy
print("Best Parameters: ", grid_search.best_params_)


Best Parameters:  {'algorithm': 'brute', 'metric': 'cosine', 'n_neighbors': 5, 'radius': 1}


c:\Users\LAPTOP\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_search.py:979: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


In [24]:
my_n_neighbors = grid_search.best_params_['n_neighbors']
my_radius = grid_search.best_params_['radius']
my_metric = grid_search.best_params_['metric']
my_algorithm = grid_search.best_params_['algorithm']

## Sử dụng thư viện scikit-learn để tạo một mô hình KNN dựa trên ma trận thưa thớt.
## Mô hình này sẽ được sử dụng để tìm các bộ phim tương tự dựa trên độ tương đồng cosine giữa chúng.


In [25]:
knn_movie_model = NearestNeighbors(n_neighbors= my_n_neighbors, radius= my_radius, metric=my_metric, algorithm=my_algorithm)
knn_movie_model.fit(movie_to_user_sparse_df)

NearestNeighbors(algorithm='brute', metric='cosine', radius=1)

In [26]:
# hàm này lấy các chuỗi kí tự trong tên phim được nhập vào, 
# so sánh với danh sách phim 
# để tìm các phim mà có thể bạn  đang tìm kiếm
def get_possible_movies(movie):

    temp = ''
    possible_movies = case_insensitive_movies_list.copy()
    for i in movie :
      out = []
      temp += i
      for j in possible_movies:
        if temp in j:
          out.append(j)
      if len(out) == 0:
          return possible_movies
      out.sort()
      possible_movies = out.copy()

    return possible_movies

In [27]:
len(movies_list)-1

1663

In [28]:

def get_similar_movies(movie, n=10):
    index = movie_dict[movie]
    knn_input = np.asarray([movie_to_user_df.values[index]])
    n = min(len(movies_list)-1, n)
    distances, indices = knn_movie_model.kneighbors(knn_input, n_neighbors=n+1)
    # Tính toán độ tương đồng cosine từ khoảng cách cosine.
    cosine_similarities = 1 - distances.flatten()
    recommendations = [
        f"{movies_list[indices[0][i]]} - Similarity: {cosine_similarities[i]:.2f}"
        for i in range(1, len(cosine_similarities))
    ]

    return recommendations

In [29]:
# hàm trả về tên các bộ phim đề xuất dựa vào từ khóa nhập vào
def recommend():
    try:
        movie_name = entry_movie.get()
        movie_name_lower = movie_name.lower()
        if movie_name_lower not in case_insensitive_movies_list:
            raise InvalidInput
        else:
            num_recom = int(entry_num_recom)
            recommendations = get_similar_movies(movies_list[case_insensitive_movies_list.index(movie_name_lower)], num_recom)
            result_label.config(text="\n".join(recommendations))
    except InvalidInput:
        possible_movies = get_possible_movies(movie_name_lower)
        if len(possible_movies) == len(movies_list):
            result_label.config(text="Movie name entered does not exist in the list.")
        else:
            indices = [case_insensitive_movies_list.index(i) for i in possible_movies]
            suggestions = '\n'.join([movies_list[i] for i in indices])
            result_label.config(text=suggestions)

# Tạo giao diện người dùng

In [30]:
import tkinter as tk
class InvalidInput(Exception):
    pass
# # định nghĩa số lượng phim đề xuất     
entry_num_recom = 10
root = tk.Tk()
root.title("KNN Movie Recommendation System")

label_movie = tk.Label(root, text="Enter the Movie name:")
label_movie.pack(pady=10)

entry_movie = tk.Entry(root, width=30)
entry_movie.pack(pady=10)


button_recommend = tk.Button(root, text="Search", command=recommend)
button_recommend.pack(pady=20)

result_label = tk.Label(root, text="")
result_label.pack(pady=10)
# Run the Tkinter event loop
root.mainloop()